In [1]:
import unicodedata
import unidecode
import requests
import pandas as pd
import json
import numpy as np
from urllib.request import urlopen
from bs4 import BeautifulSoup

#Output Folder Data Path
path = "output_data/"


In [2]:
country = ['AO','AG','AR','AU','AT','BS','BE','BA','BR','BG','CM','CA','CV','CN','CO',
        'HR','CU','CZ','CD','DK','DM','DO','EG','EE','FI','FR','GF','GA','GE','DE',
        'GH','GR','GP','GN','GY','HT','HU','IS','IE','IR','IL','IT','JM','JP','LV','LB',
        'LT','LU','ML','MQ','MX','ME','MA','NL','NZ','NG','NO','PA','PL','PT','PR','KR','MK',
        'CG','RO','RU','LC','VC','SN','RS','SK','SI','ZA','SS','ES','SD','SE','CH','TW',
        'TT','TN','TR','US','VI','UA','GB','TZ','UY','VE']


In [3]:
us_state = ['AL','AK','AS','AZ','AR','CA','CO','CT','DE','DC','FL',
        'GA','GU','HI','ID','IL','IN','IA','KS','KY','LA','ME',
        'MD','MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ',
        'NM','NY','NC','ND','MP','OH','OK','OR','PA','PR','RI',
        'SC','SD','TN','TX','UT','VT','VA','VI','WA','WV','WI','WY']

In [4]:
letters = ["a","b","c","d","e","f","g","h","i","j","k","l","m","n","o","p","q","r","s","t","u","v","w","x","y","z"]

In [5]:
def get_player_info_byOrigin(country, state, dataframe):
    
    headers = []
    rows_data = []
    
    url = f'https://www.basketball-reference.com/friv/birthplaces.fcgi?country={co}&state={st}'

    html = urlopen(url)
    soup = BeautifulSoup(html, features='lxml')
    
    
    try:
        headers = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]
        rows = soup.findAll('tr')[2:]
        rows_data = [[td.getText() for td in rows[i].findAll('td')]for i in range(len(rows))]
    
        last_year = 2023
        for i in range(0, len(rows_data)):
            rows_data[i].insert(0, last_year)
            last_year -=1

        df = pd.DataFrame(rows_data, columns = headers)
        df['Country'] = co
        df['State'] = st

        frames=[dataframe,df]   
        dataframe_raw = pd.concat((frames),ignore_index=True).reset_index(drop=True)
        
        dataframe_raw = dataframe_raw.astype({'Player':'string'})
        dataframe_raw['Player'].str.strip()
        
        dataframe = dataframe_raw.loc[dataframe_raw['Player'] != '']
                    
        print(f'{co} , {st}')
    except Exception as e:
        print(f'{co} , {st}, {e} | Skipping...')
        pass
  
    return dataframe

In [6]:
player_info_origin = pd.DataFrame()

print(f'Retrieving Data')
print(f'---------------------')
    
for co in country:
       
    if co == 'US':
        for st in us_state:
            player_info_origin = get_player_info_byOrigin(co, st, player_info_origin)
            
    else:
        st = ''
        player_info_origin = get_player_info_byOrigin(co, st, player_info_origin)
        
player_info_origin['Player'] = player_info_origin['Player'].map(lambda x: x.rstrip('*'))
        
print(f'---------------------Data Complete---------------------')


player_info_origin.to_csv(f'{path}bkr_players_info_ByOrigin.csv', encoding='utf-8-sig')



Retrieving Data
---------------------
AO , 
AG , 
AR , 
AU , 
AT , 
BS , 
BE , 
BA , 
BR , 
BG , 
CM , 
CA , 
CV , 
CN , 
CO , 
HR , 
CU , 
CZ , 
CD , 
DK , 
DM , 
DO , 
EG , 
EE , 
FI , 
FR , 
GF , 
GA , 
GE , 
DE , 
GH , 
GR , 
GP , 
GN , 
GY , 
HT , 
HU , 
IS , 
IE , 
IR , 
IL , 
IT , 
JM , 
JP , 
LV , 
LB , 
LT , 
LU , 
ML , 
MQ , 
MX , 
ME , 
MA , 
NL , 
NZ , 
NG , 
NO , 
PA , 
PL , 
PT , 
PR , 
KR , 
MK , 
CG , 
RO , 
RU , 
LC , 
VC , 
SN , 
RS , 
SK , 
SI , 
ZA , 
SS , 
ES , 
SD , 
SE , 
CH , 
TW , 
TT , 
TN , 
TR , 
US , AL
US , AK
US , AS, list index out of range | Skipping...
US , AZ
US , AR
US , CA
US , CO
US , CT
US , DE
US , DC
US , FL
US , GA
US , GU, list index out of range | Skipping...
US , HI
US , ID
US , IL
US , IN
US , IA
US , KS
US , KY
US , LA
US , ME
US , MD
US , MA
US , MI
US , MN
US , MS
US , MO
US , MT
US , NE
US , NV
US , NH
US , NJ
US , NM
US , NY
US , NC
US , ND
US , MP, list index out of range | Skipping...
US , OH
US , OK
US , OR
US , PA
US , PR, list ind

In [7]:
  
def get_player_info_byLetter(letter, dataframe):
    
    headers = []
    rows_data = []
    
 
    url = f'https://www.basketball-reference.com/players/{letter}/'

    html = urlopen(url)
    soup = BeautifulSoup(html, features='lxml')
    
    
    try:
        headers = [th.getText() for th in soup.findAll('tr', limit=2)[0].findAll('th')]
        rows = soup.findAll('tr')[1:]
        players = [[td.getText() for td in rows[i].findAll('th')][0] for i in range(len(rows))]
        rows_data = [[td.getText() for td in rows[i].findAll('td')]for i in range(len(rows))]
        references = [[td.get('href') for td in rows[i].findAll('a')] for i in range(len(rows))]
 
        headers.append("HTML Reference")
   
        for i in range(0, len(rows_data)):
            rows_data[i].insert(0, players[i])
            rows_data[i].append(references[i][0])

        df = pd.DataFrame(rows_data, columns = headers)

        frames=[dataframe,df]   
        dataframe_raw = pd.concat((frames),ignore_index=True).reset_index(drop=True)
        
        dataframe_raw = dataframe_raw.astype({'Player':'string'})
        dataframe_raw['Player'].str.strip()
        dataframe = dataframe_raw.loc[dataframe_raw['Player'] != '']
        
        print(f'{letter}')
    except Exception as e:
        print(f'{letter}, {e} | Skipping...')
        pass
    return dataframe
    


In [8]:
player_info_letter = pd.DataFrame()

print(f'Retrieving Data')
print(f'---------------------')
    
for let in letters:
       
            player_info_letter = get_player_info_byLetter(let, player_info_letter)

player_info_letter['Player'] = player_info_letter['Player'].map(lambda x: x.rstrip('*'))

print(f'---------------------Data Complete---------------------')

player_info_letter.to_csv(f'{path}bkr_players_info_ByLetter.csv', encoding='utf-8-sig')

Retrieving Data
---------------------
a
b
c
d
e
f
g
h
i
j
k
l
m
n
o
p
q
r
s
t
u
v
w
x
y
z
---------------------Data Complete---------------------


In [9]:
test_data=pd.DataFrame()

headers = []
rows_data = []
    
 
url = f'https://www.basketball-reference.com/players/a/'

html = urlopen(url)
soup = BeautifulSoup(html, features='lxml')
    
    
try:
    headers = [th.getText() for th in soup.findAll('tr', limit=2)[0].findAll('th')]
    rows = soup.findAll('tr')[1:]
    players = [[td.getText() for td in rows[i].findAll('th')][0] for i in range(len(rows))]
    reference = [[td.get('href') for td in rows[i].findAll('a')] for i in range(len(rows))]
    rows_data = [[td.getText() for td in rows[i].findAll('td')]for i in range(len(rows))]
    
    for i in range(0, len(rows_data)):
        rows_data[i].insert(0, players[i])

    df = pd.DataFrame(rows_data, columns = headers)

    frames=[test_data,df]   
    test_data_raw = pd.concat((frames),ignore_index=True).reset_index(drop=True)

    test_data_raw = test_data_raw.astype({'Player':'string'})
    test_data_raw['Player'].str.strip()
        
    test_data = test_data_raw.loc[test_data_raw['Player'] != '']

except Exception as e:
    print(e)
    pass

print(reference[0][0])

/players/a/abdelal01.html
